In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

Passos desse notebook

1. Merge de datasets

2. Transformar colunas

(Deixamos apenas os projetos de Lei q tem partido)

- Variáveis Categóricas (Temas)
    - One-Hot Encoding
- Variáveis Numéricas (Porcentagem de deputados do partido que ta propondo o PL)
    - Normalizar

nome_PL | 5 categorias | porcentagem deputados

Vão ser 7 colunas

In [2]:
ListaProposicoes = os.path.join('data', 'ListaProposicoes_2013_2024.csv')
ListaProposicoesVotadas_Plenario = os.path.join('data', 'ListaProposicoesVotadasPlenario_2013_2024.csv')
ObterProposicoes = os.path.join('data', 'ObterProposicoes_2013_2024.csv')
ObterProposicoes_2013 = os.path.join('data', 'obter_proposicoes' ,'ObterProposicoes_2013.csv')
VotacoesProposicoes = os.path.join('data', 'VotacoesProposicoes_2013_2024.csv')

In [3]:
df = pd.read_csv(ObterProposicoes)
df_pvp = pd.read_csv(ListaProposicoesVotadas_Plenario)
df_vp = pd.read_csv(VotacoesProposicoes)
df_lp = pd.read_csv(ListaProposicoes)

In [4]:
df.columns

Index(['nomeProposicao', 'idProposicao', 'idProposicaoPrincipal',
       'nomeProposicaoOrigem', 'tipoProposicao', 'tema', 'Ementa',
       'ExplicacaoEmenta', 'Autor', 'ideCadastro', 'ufAutor', 'partidoAutor',
       'DataApresentacao', 'RegimeTramitacao', 'UltimoDespacho', 'Apreciacao',
       'Indexacao', 'Situacao', 'LinkInteiroTeor', 'apensadas',
       'apensadas_proposicao'],
      dtype='object')

In [5]:
print("Quantidade de linhas em df: ", df.shape[0])
print("Quantidade de linhas em df_lp: ", df_lp.shape[0])  

print("Quantidade de colunas em df: ", df.shape[1])
print("Quantidade de colunas em df_lp: ", df_lp.shape[1]) 

Quantidade de linhas em df:  132399
Quantidade de linhas em df_lp:  39701
Quantidade de colunas em df:  21
Quantidade de colunas em df_lp:  31


In [6]:
len(df)

132399

In [7]:
df.idProposicao.nunique()

39701

In [8]:
# Imprimir valores duplicados na coluna 'idProposicao'
duplicated_ids = df[df['idProposicao'].duplicated()]['idProposicao']
print(duplicated_ids)

12832     605259
12833     605279
12834     605290
12835     605291
12836     605307
           ...  
132394    574182
132395    569556
132396    568561
132397    568597
132398    565698
Name: idProposicao, Length: 92698, dtype: int64


## Removendo duplicatas

Devido ao processamento paralelo das requisições, alguns dados foram salvos mais de uma vez com base no ID. Para garantir a integridade dos dados, as duplicatas serão eliminadas.


In [9]:
duplicated_ids = df[df['idProposicao'].duplicated()]['idProposicao']
#print(duplicated_ids)

if duplicated_ids.any():
    df = df.drop_duplicates(subset='idProposicao')

#print(df)

In [10]:
df.columns

Index(['nomeProposicao', 'idProposicao', 'idProposicaoPrincipal',
       'nomeProposicaoOrigem', 'tipoProposicao', 'tema', 'Ementa',
       'ExplicacaoEmenta', 'Autor', 'ideCadastro', 'ufAutor', 'partidoAutor',
       'DataApresentacao', 'RegimeTramitacao', 'UltimoDespacho', 'Apreciacao',
       'Indexacao', 'Situacao', 'LinkInteiroTeor', 'apensadas',
       'apensadas_proposicao'],
      dtype='object')

In [11]:
df_tran = df[df.Situacao.str.contains('Tran')]
df_tran

,nomeProposicao,idProposicao,idProposicaoPrincipal,nomeProposicaoOrigem,tipoProposicao,tema,Ementa,ExplicacaoEmenta,Autor,ideCadastro,...,partidoAutor,DataApresentacao,RegimeTramitacao,UltimoDespacho,Apreciacao,Indexacao,Situacao,LinkInteiroTeor,apensadas,apensadas_proposicao
3,PL 7027/2013,605291,\n,\n,Projeto de Lei,"Política, Partidos e Eleições",Cria cargos efetivos e funções comissionadas n...,\n,TRIBUNAL SUPERIOR ELEITORAL,\n,...,\n,20/12/2013,"Prioridade (Art. 151, II, RICD)","Às Comissões de Trabalho, de Administração e S...",Proposição Sujeita à Apreciação Conclusiva pel...,"Criação, cargo efetivo, função de confiança, T...",Tranformada no(a) Lei Ordinária 13150/2015,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
31,PL 6999/2013,604844,\n,\n,Projeto de Lei,"Direito Penal e Processual Penal; Indústria, C...",Dispõe sobre o crime de abigeato e sobre o com...,"Altera o Decreto-lei nº 2.848, de 1940 e a Lei...",Afonso Hamm,136811,...,PP,18/12/2013,"Urgência (Art. 155, RICD)",À Comissão de Constituição e Justiça e de Cida...,Proposição Sujeita à Apreciação do Plenário,"Alteração, Código Penal, Lei dos Crimes Contra...",Tranformada no(a) Lei Ordinária 13330/2016,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
32,PL 6998/2013,604836,\n,\n,Projeto de Lei,Direitos Humanos e Minorias,Altera o art. 1º e insere dispositivos sobre a...,\n,Osmar Terra,73692,...,PMDB,18/12/2013,"Ordinário (Art. 151, III, RICD)","Revejo o despacho aposto ao PL 6.998/13, para ...",Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, Estatuto da Criança e do Adolescent...",Tranformada no(a) Lei Ordinária 13257/2016,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
173,PL 6854/2013,602755,\n,PLS 412/2008,Projeto de Lei,Direitos Humanos e Minorias,"Altera a Lei nº 8.069, de 13 de julho de 1990 ...",\n,Senado Federal - Flávio Arns,\n,...,PSDB,29/11/2013,"Prioridade (Art. 151, II, RICD)",Às Comissões de Educação e Constituição e Just...,Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, Estatuto da Criança e do Adolescent...",Tranformada no(a) Lei Ordinária 13306/2016,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
174,PL 6856/2013,602757,\n,PLS 680/2011,Projeto de Lei,Educação,"Altera o art. 14 da Lei nº 11.947, de 16 de ju...",\n,Senado Federal - Ana Rita,\n,...,PT,29/11/2013,"Prioridade (Art. 151, II, RICD)","Às Comissões de Agricultura, Pecuária, Abastec...",Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, Lei da Alimentação Escolar, aquisiç...",Tranformada no(a) Lei Ordinária 14660/2023,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73072,PL 914/2024,2422697,\n,\n,Projeto de Lei,"Ciência, Tecnologia e Inovação; Finanças Públi...",Institui o Programa Mobilidade Verde e Inovaçã...,\n,Poder Executivo,\n,...,\n,21/03/2024,"Urgência (Art. 155, RICD)",EMENDAS DO SENADO FEDRAL: À Comissão de Especi...,Proposição Sujeita à Apreciação do Plenário,"Criação, Programa Mobilidade Verde e Inovação ...",Tranformada no(a) Lei Ordinária 14902/2024,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
73369,PL 545/2024,2419197,\n,\n,Projeto de Lei,"Administração Pública; Indústria, Comércio e S...","Altera a Lei nº 14.002, de 22 de maio de 2020,...",\n,José Guimarães,141470,...,PT,05/03/2024,"Urgência (Art. 155, RICD)",Às Comissões de \nRelações Exteriores e de Def...,Proposição Sujeita à Apreciação do Plenário,"Alteração, Legislação, Agência Brasileira de P...",Tranformada no(a) Lei Ordinária 14901/2024,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,NaN
73713,PL 147/2024,2417234,\n,\n,Projeto de Lei,Direitos Humanos e Minorias,"Altera a Lei n. 14.448, de 9 de setembro de 20...",\n,Maria Arraes,220668,...,SOLIDARI,06/02/2024,"Urgência (Art. 155, RICD)",Às Comissões de \nDefesa dos Direitos da Mulhe...,Proposição Sujeita à Apreciação do Plenário,"Alteração, Lei Federal, criação, Projeto Banc

In [12]:
df['DataApresentacao'] = pd.to_datetime(df['DataApresentacao'], format='%d/%m/%Y')

In [13]:
df.DataApresentacao.value_counts().sort_index()

DataApresentacao
2013-02-04    13
2013-02-05    20
2013-02-06    11
2013-02-07     5
2013-02-08     1
              ..
2024-10-09    14
2024-10-10    33
2024-10-11    12
2024-10-14    14
2024-10-15    24
Name: count, Length: 2344, dtype: int64

dv_vp

Tem algo errado.


In [14]:
df_vp.nomeproposicao.value_counts()

nomeproposicao
PL 3729/2004                                 2772
PL 8939/2017                                 2700
PL 3261/2019                                 2592
PL 5029/2019 (Nº Anterior: PL 11021/2018)    2171
PL 5735/2013                                 2140
                                             ... 
PL 1847/2024                                   11
PL 3/2024                                       9
PL 858/2024                                     8
PL 1213/2024                                    5
PL 3117/2024                                    4
Name: count, Length: 243, dtype: int64

In [15]:
df_vp

,resumo,data,hora,objvotacao,codsessao,nomeproposicao,bancada_sigla,bancada_orientacao,deputado_nome,deputado_idecadastro,deputado_partido,deputado_uf,deputado_voto
0,Mantido o texto. Sim: 268; não: 62; abstenção:...,9/7/2013,23:02,DVS - DEM - § 2º DO ART. 23 DO SUBSTITUTIVO DO...,4976,PL 4529/2004,Minoria,Liberado,Pedro Guerra,174314,PSD,PR,Sim
1,Mantido o texto. Sim: 268; não: 62; abstenção:...,9/7/2013,23:02,DVS - DEM - § 2º DO ART. 23 DO SUBSTITUTIVO DO...,4976,PL 4529/2004,Minoria,Liberado,Pedro Guerra,174314,PSD,PR,Sim
2,Rejeitado o Requerimento. Sim: 2; não: 270; ab...,5/11/2013,20:53,REQUERIMENTO DE RETIRADA DE PAUTA,5097,PL 6025/2005,Repr.PMN,Não,Amir Lando,174861,PMDB,RO,Não
3,Mantido o parágrafo. Sim: 206; não: 159; abste...,4/2/2014,21:30,DVS - PP - § 19 DO ART. 85 DA EMENDA AGLUTINATIVA,5151,PL 6025/2005,GOV.,Não,Helcio Silva,175737,PT,SP,Não
4,Aprovada a Emenda Aglutinativa nº 7. Sim: 279;...,11/2/2014,20:57,DVS - PTB - EMENDA AGLUTINATIVA Nº 7,5160,PL 6025/2005,GOV.,Não,Helcio Silva,175737,PT,SP,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104181,Mantido o texto. Sim: 206; Não: 70; Total: 276.,11/9/2024,00:26,DTQ. 4 - PL - ARTIGO 43.,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104182,Mantido o texto. Sim: 200; Não: 80; Abstenção:...,11/9/2024,00:36,DTQ. 2 - PL - ARTIGO 44.,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104183,Mantido o texto. Sim: 189; Não: 140; Total: 329.,11/9/2024,00:55,DTQ. 3 - PL - CAPÍTULO VIII,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104184,Rejeitada a Emenda de Plenário nº 18. Sim: 133...,11/9/2024,00:13,DTQ. 5 - PL - EMENDA DE PLENÁRIO Nº 18,17505,PL 1847/2024,Governo,Não,Newton Bonin,229939,União,PR,Não


Colunas selecionadas para ficar:

In [16]:
df = df[["nomeProposicao", "idProposicao", "tema", "partidoAutor", "DataApresentacao", "RegimeTramitacao","Situacao"]]

In [17]:
df

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,RegimeTramitacao,Situacao
0,PL 7024/2013,605259,Administração Pública,\n,2013-12-20,"Prioridade (Art. 151, II, RICD)",MESA - Arquivada
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,2013-12-20,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,2013-12-20,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto
3,PL 7027/2013,605291,"Política, Partidos e Eleições",\n,2013-12-20,"Prioridade (Art. 151, II, RICD)",Tranformada no(a) Lei Ordinária 13150/2015
4,PL 7029/2013,605307,Educação,PT,2013-12-23,"Ordinário (Art. 151, III, RICD)",CFT - Aguardando Designação de Relator(a)
...,...,...,...,...,...,...,...
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,2024-01-10,"Prioridade (Art. 151, II, RICD)",MESA - Aguardando Apreciação pelo Senado Federal
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,2024-02-02,"Prioridade (Art. 151, II, RICD)",CDE - Aguardando Parecer
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,2024-02-05,"Prioridade (Art. 151, II, RICD)",CPASF - Tramitando em Conjunto
73841,PL 2/2024,2416804,Economia; Finanças Públicas e Orçamento; Indús...,\n,2024-01-03,"Urgência (Art. 64, CF)",Tranformada no(a) Lei Ordinária 14871/2024


In [18]:
primeiras_palavras = df['Situacao'].str.split().str[0].unique()

In [19]:
primeiras_palavras

array(['MESA', 'CCJC', 'Tranformada', 'CFT', 'PLEN', 'CSAUDE', 'CPASF',
       'Diversas', 'CE', 'CASP', 'CTRAB', 'CAPADR', 'CDC', 'CDHMIR',
       'CME', 'CCP', 'CMULHER', 'CPOVOS', 'CMADS', 'CDE', 'CVT',
       'ARQUIVO', 'SERAU(SGM)', 'CREDN', 'SGM', 'CCOM', 'MESA-CN', 'CCTI',
       'CICS', 'CDU', 'GTCL', 'CCULT', 'CPD', 'Originou', 'NAJUR(SGM)',
       'CTUR', 'CINDRE', 'CESPO', 'CSPCCO', '1SECM', 'SECAP(SGM)',
       'SGMGAB', 'CIDOSO', 'SIDEX', 'CD'], dtype=object)

In [20]:
df_diversas = df[df['Situacao'].str.contains('Diversas')]

In [21]:
df_diversas

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,RegimeTramitacao,Situacao
19,PL 7010/2013,604927,Direito Penal e Processual Penal; Meio Ambient...,PTB,2013-12-18,"Ordinário (Art. 151, III, RICD)",Diversas
49,PL 6980/2013,604628,Saúde,PDT,2013-12-17,"Prioridade (Art. 151, II, RICD)",Diversas
65,PL 6964/2013,604508,Cidades e Desenvolvimento Urbano,\n,2013-12-16,"Prioridade (Art. 151, II, RICD)",Diversas
87,PL 6940/2013,604138,Educação,PSD,2013-12-11,"Prioridade (Art. 151, II, RICD)",Diversas
94,PL 6935/2013,604123,Trabalho e Emprego,PSDB,2013-12-11,"Prioridade (Art. 151, II, RICD)",Diversas
...,...,...,...,...,...,...,...
73771,PL 83/2024,2417028,"Arte, Cultura e Religião; Homenagens e Datas C...",PCdoB,2024-02-05,.,Diversas
73794,PL 62/2024,2416975,"Arte, Cultura e Religião; Homenagens e Datas C...",PCdoB,2024-02-05,.,Diversas
73796,PL 54/2024,2416941,Defesa e Segurança; Direito Penal e Processual...,PL,2024-02-05,.,Diversas
73819,PL 26/2024,2416879,Direitos Humanos e Minorias,UNIÃO,2024-02-05,"Prioridade (Art. 151, II, RICD)",Diversas


In [23]:
df_diversas.tema.unique()

array(['Direito Penal e Processual Penal; Meio Ambiente e Desenvolvimento Sustentável',
       'Saúde', 'Cidades e Desenvolvimento Urbano', 'Educação',
       'Trabalho e Emprego', 'Indústria, Comércio e Serviços',
       'Administração Pública; Direitos Humanos e Minorias',
       'Cidades e Desenvolvimento Urbano; Educação',
       'Direito e Defesa do Consumidor; Saúde',
       'Comunicações; Direito e Defesa do Consumidor',
       'Direito e Defesa do Consumidor; Direitos Humanos e Minorias; Indústria, Comércio e Serviços',
       'Economia', 'Direito e Justiça', 'Arte, Cultura e Religião',
       'Direito e Defesa do Consumidor',
       'Previdência e Assistência Social',
       'Direito Penal e Processual Penal',
       'Finanças Públicas e Orçamento; Meio Ambiente e Desenvolvimento Sustentável',
       'Homenagens e Datas Comemorativas; Saúde',
       'Direitos Humanos e Minorias', 'Defesa e Segurança',
       'Viação, Transporte e Mobilidade',
       'Homenagens e Datas Comemor

In [24]:
unique_values = df_diversas['tema'].str.split('; ').explode().unique()

print(unique_values)

['Direito Penal e Processual Penal'
 'Meio Ambiente e Desenvolvimento Sustentável' 'Saúde'
 'Cidades e Desenvolvimento Urbano' 'Educação' 'Trabalho e Emprego'
 'Indústria, Comércio e Serviços' 'Administração Pública'
 'Direitos Humanos e Minorias' 'Direito e Defesa do Consumidor'
 'Comunicações' 'Economia' 'Direito e Justiça' 'Arte, Cultura e Religião'
 'Previdência e Assistência Social' 'Finanças Públicas e Orçamento'
 'Homenagens e Datas Comemorativas' 'Defesa e Segurança'
 'Viação, Transporte e Mobilidade' 'Direito Civil e Processual Civil'
 'Energia, Recursos Hídricos e Minerais'
 'Relações Internacionais e Comércio Exterior' 'Esporte e Lazer'
 'Agricultura, Pecuária, Pesca e Extrativismo'
 'Processo Legislativo e Atuação Parlamentar'
 'Política, Partidos e Eleições' '\n  ' 'Ciência, Tecnologia e Inovação'
 'Estrutura Fundiária' 'Turismo' 'Direito Constitucional']


In [25]:
len(unique_values)

31

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Listas das categorias
direitos_justica = [
    'Direito Penal e Processual Penal', 'Direito e Justiça', 
    'Direito Civil e Processual Civil', 'Direito Constitucional', 
    'Direito e Defesa do Consumidor', 'Direitos Humanos e Minorias'
]

desenvolvimento_social_urbano = [
    'Cidades e Desenvolvimento Urbano', 'Saúde', 'Educação', 
    'Trabalho e Emprego', 'Arte, Cultura e Religião', 
    'Esporte e Lazer', 'Previdência e Assistência Social', 
    'Homenagens e Datas Comemorativas'
]

economia_desenvolvimento_sustentavel = [
    'Meio Ambiente e Desenvolvimento Sustentável', 'Economia', 
    'Indústria, Comércio e Serviços', 'Finanças Públicas e Orçamento', 
    'Agricultura, Pecuária, Pesca e Extrativismo', 'Estrutura Fundiária', 
    'Turismo', 'Relações Internacionais e Comércio Exterior', 
    'Energia, Recursos Hídricos e Minerais'
]

administracao_politicas_publicas = [
    'Administração Pública', 'Processo Legislativo e Atuação Parlamentar', 
    'Política, Partidos e Eleições', 'Defesa e Segurança', 
    'Comunicações', 'Viação, Transporte e Mobilidade', 
    'Ciência, Tecnologia e Inovação'
]

infraestrutura_gestao_recursos = [
    'Viação, Transporte e Mobilidade', 'Energia, Recursos Hídricos e Minerais', 
    'Estrutura Fundiária', 'Comunicações'
]

# Função para categorizar temas
def categorizar_tema(tema):
    if tema in direitos_justica:
        return 'Direitos e Justiça'
    elif tema in desenvolvimento_social_urbano:
        return 'Desenvolvimento Social e Urbano'
    elif tema in economia_desenvolvimento_sustentavel:
        return 'Economia e Desenvolvimento Sustentável'
    elif tema in administracao_politicas_publicas:
        return 'Administração e Políticas Públicas'
    elif tema in infraestrutura_gestao_recursos:
        return 'Infraestrutura e Gestão de Recursos'
    else:
        return 'Outros'

# Supondo que você tenha um dataframe df com a coluna 'tema'
df['categoria'] = df['tema'].apply(categorizar_tema)

# Realizando o One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(df[['categoria']])

# Convertendo para dataframe e adicionando ao original
categorias_encoded = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.categories_[0])
df = pd.concat([df, categorias_encoded], axis=1)

# Exibe o dataframe resultante
print(df)
